In [ ]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install PyMuPDF
!pip install langchain_openai
!pip install pinecone
!pip install langchain_pinecone
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: pinecone-plugin-inference
    Found existing installation: pinecone-plugin-inference 2.0.1
    Uninstalling pinecone-plugin-inference-2.0.1:
      Successfully uninstalled pinecone-plugin-inference-2.0.1
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.2
    Uninstalling aiohttp-3.11.2:
      Successfully uninstalled

In [ ]:
#讀取檔案
from pprint import pprint
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyMuPDFLoader("/content/drive/MyDrive/rag_pdf/FD.pdf")
documents = loader.load_and_split(
    text_splitter=RecursiveCharacterTextSplitter(
        chunk_size=300,
        chunk_overlap=50
    )
)

In [ ]:
#寫入向量資料庫
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
import pinecone
embedding = OpenAIEmbeddings(
    openai_api_key=OPENAI_API_KEY
)

vector_store = PineconeVectorStore(
    index=Pinecone(api_key=PINECONE_API_KEY).Index("rag"),
    embedding=embedding
)
vector_store.add_documents(loader.load())

['97539c9d-601d-4ea9-becb-77c3d944e8bf',
 '6b6ef3ec-0b6d-4b63-b4ed-c69475b88b56',
 'a5e8d6f8-c758-4730-8b36-aae76e263903',
 '5f56cbc2-8e87-4962-b65d-75aaf9d36c41',
 '190d2bbf-af97-4e8f-bae8-7124733d7e7c',
 'ecf94fe2-7c55-4578-9eef-d49086896518',
 'dbb1a32b-06ae-4ccc-814a-88a50e2e8215',
 'a19b2de5-788c-437c-9b09-cdabc7895e43',
 '3257176c-eb1b-47b6-9d53-f54741535ec3',
 '3f8d2a90-9038-4697-bd62-33d1edc61ceb',
 'eb35b86d-c81b-4c36-a155-1081abaab458',
 '8488fb75-bca8-4f00-988c-18d5b9917b7f',
 'e3d54466-dce1-496e-95e9-9525c795e462',
 '13342335-f562-4ca4-9b73-72dd4b94de17',
 '3346c9ab-6c37-43c6-939b-0299823381b1',
 '6a4d2657-fcb7-4f73-8f66-8586615db7b6',
 '944d8388-f3e6-4f1b-b166-15a97bb1824c',
 'd69e135b-e41a-4542-a785-bd807cbc2f44',
 'ee7c8d58-9e45-4ae2-b162-b08927922368',
 '2d934f21-efbb-44a0-a6ba-7883bdea3928',
 'fbb71da8-3b0e-4b18-aa57-1c440438a682',
 '2b50cae6-9120-468c-b26e-5f1bc8a07c3f',
 'c2270513-f344-4fcb-a7cb-1f73adf94739',
 'd0b1ef8f-6a0f-415c-a3b7-c905395a0f34',
 '3ac8feb4-f116-

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.memory import ConversationSummaryBufferMemory
from langchain_core.output_parsers import BaseOutputParser
from typing import List
import logging
from langchain_core.prompts import PromptTemplate

# 定義 LineListOutputParser 來解析多行文本
class LineListOutputParser(BaseOutputParser[List[str]]):
    """Output parser for a list of lines."""

    def parse(self, text: str) -> List[str]:
        lines = text.strip().split("\n")
        return list(filter(None, lines))  # Remove empty lines

# 設置日誌級別，顯示生成的查詢
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)
# 使用 ChatOpenAI 作為 LLM 來生成查詢
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    model_name="gpt-4o"
)

# 生成自定義 PromptTemplate
CUSTOM_QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""你是一個 AI 語言模型助手。你的任務是根據用戶輸入的問題生成三個不同版本的查詢，以繁體中文進行檢索。請將這些查詢分行顯示。
    原始問題: {question}"""
)
# 建立 MultiQueryRetriever，並將 Pinecone 索引作為檢索器
llm_chain = CUSTOM_QUERY_PROMPT | llm | LineListOutputParser()
retriever_from_llm = MultiQueryRetriever(
    retriever=vector_store.as_retriever(),
    llm_chain=llm_chain
)
question = "指標"
unique_docs = retriever_from_llm.invoke(question)
unique_docs

INFO:langchain.retrievers.multi_query:Generated queries: ['1. 指標的定義和用途', '2. 常見的經濟指標有哪些', '3. 如何分析和解讀指標數據']


[Document(id='64798b3f-81b2-4f7e-be39-e3635c6e791c', metadata={'author': 'A000000', 'creationDate': "D:20240417093840+08'00'", 'creator': 'Microsoft® Word 2019', 'file_path': '/content/drive/MyDrive/LineBot/FD.pdf', 'format': 'PDF 1.7', 'keywords': '', 'modDate': "D:20240417093840+08'00'", 'page': 41.0, 'producer': 'Microsoft® Word 2019', 'source': '/content/drive/MyDrive/LineBot/FD.pdf', 'subject': '', 'title': '「西醫基層與醫院之財務風險分擔監控指標」會議議程', 'total_pages': 42.0, 'trapped': ''}, page_content='42 \n \n治療指引 \n參考指標 \n分子定義 \n分母定義 \n操作型定義 \n \nABC 達標率 \n \n \n \n\uf0acStark, C.S., Fradkin, J.E., \nSaydah, S.H., Rust, K.F., \nCowie, C.C. (2013).The \nprevalence of meeting A1C, \nblood pressure, and LDL goals \namong people with diabetes, \n1988-2010. Diabetes \nCare,36(8),2271-9. doi: \n10.2337/dc12-2258.  \n\uf0acAli, M.K., Bullard, K.M., \nGregg, E.W., Del, R.C. (2014). \nA cascade of care for diabetes \nin the United States: \nvisualizing the gaps. Ann \nIntern Med, 161(10),681-9. \ndoi: 10.

In [ ]:
def qa(question: str, k: int = 1):
    documents = vector_store.similarity_search(question, k=k)
    prompt = ChatPromptTemplate.from_template(
    """
    原始問題: {question}
    取得的參考資料: {data}
    "你是專業客服人員,請根據上下文來回答問題,"
    "你不知道答案就說你不知道, 不要試圖編造答案。\n"
    請用繁體中文回答
    """
    )
    llm = ChatOpenAI(
        api_key=OPENAI_API_KEY,
        model_name="gpt-4o"
    )
    chain = prompt | llm

    result = chain.invoke({"question": question, "data": documents[0].page_content})
    return str(result.content)

In [ ]:
qa("指標")

'請問您有什麼具體問題需要我協助解答嗎？根據您提供的內容，這段文字似乎是在描述糖尿病患者的治療指引及相關達標率的定義。如果您有任何具體的問題或需要進一步的資訊，請隨時告訴我！'